# Giovanni Averaging Service Regression Test Suite

This notebook contains a suite of regression tests for the Giovanni Averaging Service, to ensure the service continues function with all Harmony updates. The tests include a test for an area averaged time series and a time averaged map.

## Prerequisites

The dependencies for this notebook are listed in the [environment.yaml](./environment.yaml). To test or install locally, create the papermill environment used in the automated regression testing suite:

`conda env create -f ./environment.yaml && conda activate papermill-giovanni-averaging-service`

A `.netrc` file must also be located in the `test` directory of this repository.

## Import Requirements

In [1]:
from harmony import Dimension, Client, Collection, Request, Environment
import datetime as dt
import os
import tempfile
from util import assert_csv_equal, assert_geotiff_equal

## Set Default Parameters

`papermill` requires default values for parameters used on the workflow. In this case, `harmony_host_url`.

In [2]:
harmony_host_url = 'https://harmony.uat.earthdata.nasa.gov'
# harmony_host_url = 'https://harmony.sit.earthdata.nasa.gov'
# harmony_host_url = 'http://localhost:3000'
# harmony_host_url = 'https://harmony.earthdata.nasa.gov'

### Identify Harmony environment (for easier reference)

In [3]:
host_environment = {
    'http://localhost:3000': Environment.LOCAL,
    'https://harmony.sit.earthdata.nasa.gov': Environment.SIT,
    'https://harmony.uat.earthdata.nasa.gov': Environment.UAT,
    'https://harmony.earthdata.nasa.gov': Environment.PROD,
}


harmony_environment = host_environment.get(harmony_host_url)

if harmony_environment is not None:
    harmony_client = Client(env=harmony_environment)

### Identify Test Collection Ids

In [ ]:
non_prod_collection_id = Collection(id="C1240560361-GES_DISC")
prod_collection_id = Collection(id="C2723754847-GES_DISC")

if harmony_environment == Environment.PROD:
    collection = prod_collection_id
else:
    collection = non_prod_collection_id

## Run Area Averaged Time Series Test 

### Make an Area Averaged Time Series Request

In [ ]:
request = Request(
    collection=collection,
    temporal={
        "start": dt.datetime(2021, 8, 29, 7, 30),
        "stop": dt.datetime(2021, 8, 29, 9, 30),
    },
    dimensions=[Dimension("lat", 25, 35), Dimension("lon", -110, -78)],
    variables=["GPM_3IMERGHH_07_precipitation"],
    format="text/csv",
    average="area",
)

job_id = harmony_client.submit(request)

### Confirm Request Results Exist and are Expected

In [5]:
reference_csv = os.path.join(
    os.getcwd(),
    "reference_data",
    "8539508_GIOVANNI-areaAvgTimeSeries.GPM_3IMERGHH_07_precipitation.20210829-20210829.110W_25N_78W_35N.csv",
)

with tempfile.TemporaryDirectory() as tmpdir:
    results = harmony_client.download_all(job_id, directory=tmpdir)
    for r in results:
        downloaded_csv = r.result()
        assert os.path.exists(downloaded_csv)
        assert_csv_equal(downloaded_csv, reference_csv)

/var/folders/3t/h5wtz4jx5y767tbwyy0jmy_h0000gq/T/tmprk934an6/8561778_GIOVANNI-areaAvgTimeSeries.GPM_3IMERGHH_07_precipitation.20210829-20210829.110W_25N_78W_35N.csv


## Run Time Averaged Map Test

### Make a Time Averaged Map Request

In [ ]:
request = Request(
    collection=collection,
    temporal={
        "start": dt.datetime(2021, 8, 29, 7, 30),
        "stop": dt.datetime(2021, 8, 29, 9, 30),
    },
    dimensions=[Dimension("lat", 25, 35), Dimension("lon", -110, -78)],
    variables=["GPM_3IMERGHH_07_precipitation"],
    format="image/tiff",
    average="time",
)

job_id = harmony_client.submit(request)

### Confirm Request Results Exist and are Expected

In [7]:
reference_tif = os.path.join(
    os.getcwd(),
    "reference_data",
    "8539587_GIOVANNI-timeAvgMap.GPM_3IMERGHH_07_precipitation.20210829-20210829.110W_25N_78W_35N.tif",
)

with tempfile.TemporaryDirectory() as tmpdir:
    results = harmony_client.download_all(job_id, directory=tmpdir)
    for r in results:
        downloaded_tif = r.result()
        assert os.path.exists(downloaded_tif)
        assert_geotiff_equal(downloaded_tif, reference_tif)

/var/folders/3t/h5wtz4jx5y767tbwyy0jmy_h0000gq/T/tmpk_mor54i/8561779_GIOVANNI-timeAvgMap.GPM_3IMERGHH_07_precipitation.20210829-20210829.110W_25N_78W_35N.tif
